In [ ]:
import json
import os
from pyapacheatlas.auth import ServicePrincipalAuthentication
from pyapacheatlas.core import PurviewClient
from pyapacheatlas.core import AtlasEntity, AtlasProcess
from pyapacheatlas.core import AtlasAttributeDef, EntityTypeDef, RelationshipTypeDef
from pyapacheatlas.core.util import GuidTracker
from pyapacheatlas.readers import ExcelReader
from pyapacheatlas.scaffolding import column_lineage_scaffold
from pyapacheatlas.readers import ExcelConfiguration, ExcelReader

In [ ]:
auth = ServicePrincipalAuthentication(
    tenant_id = "",
    client_id = "",
    client_secret = ""
)

In [ ]:
client = PurviewClient(
    account_name = "<account_name>",
    authentication = auth
)

Create a custom entity type that represents the Azure Analysis Services Server 

In [ ]:
custom_type = EntityTypeDef(
    name="azure_analysis_services_server",
    superTypes=["azure_resource"],
    description="azure_analysis_services_server",
    typeVersion="1.0",
)

#Attributes that represents the server
custom_type.addAttributeDef(
    AtlasAttributeDef(name="tier", typeName="string", isOptional=False),
    AtlasAttributeDef(name="subscription", typeName="string", isOptional=False),
    AtlasAttributeDef(name="resource-group", typeName="string", isOptional=False),
    AtlasAttributeDef(name="location", typeName="string", isOptional=False)
)

client.upload_typedefs(
    entityDefs=[custom_type],
    force_update=True
)

Create a custom entity type that represents the Azure Analysis Services Model

In [ ]:
custom_type = EntityTypeDef(
    name="azure_analysis_services_model",
    superTypes=["Asset"],
    description="azure_analysis_services_model",
    typeVersion="1.0",
    serviceType="Azure Analysis Services Model",
)

#Attributes that represents the model
custom_type.addAttributeDef(
    AtlasAttributeDef(name="database-name", typeName="string", isOptional=False),
    AtlasAttributeDef(name="compatibility-level", typeName="int", isOptional=False)
)

client.upload_typedefs(
    entityDefs=[custom_type],
    force_update=True
)

Create a custom entity type that represents the Azure Analysis Services Model Table

In [ ]:
custom_type = EntityTypeDef(
    name="azure_analysis_services_table",
    superTypes=["DataSet"],
    description="azure_analysis_services_table",
    typeVersion="1.0",
    serviceType="Azure Analysis Services Model",
)

#Attributes that represents the model table
custom_type.addAttributeDef(
    AtlasAttributeDef(name="tableType", typeName="string", isOptional=False),
    AtlasAttributeDef(name="query", typeName="string", isOptional=False)
)

client.upload_typedefs(
    entityDefs=[custom_type],
    force_update=True
)

Create a custom entity type that represents the Azure Analysis Services Model Table Column

In [ ]:
custom_type = EntityTypeDef(
    name="azure_analysis_services_column",
    superTypes=["DataSet"],
    description="azure_analysis_services_column",
    typeVersion="1.0",
    serviceType="Azure Analysis Services Model",
)

#Attributes that represents the model column
custom_type.addAttributeDef(
    AtlasAttributeDef(name="data_type", typeName="string", isOptional=False),
    AtlasAttributeDef(name="isHidden", typeName="string", isOptional=False),
    AtlasAttributeDef(name="isKey", typeName="string", isOptional=False),
    AtlasAttributeDef(name="columnType", typeName="string", isOptional=False)
)

client.upload_typedefs(
    entityDefs=[custom_type],
    force_update=True
)

Create an Entity (AAS Server) based on that EntityTypeDef that was created before

In [ ]:
gt = GuidTracker()
serverGuid = gt.get_guid()
ae = AtlasEntity(
    name="AASHOMSERVER",
    typeName="azure_analysis_services_server",
    qualified_name="aashomserver.azure.net",
    guid=serverGuid,
    attributes={"tier":"D5", "subscription":"GProd", "resource-group":"USAZU1VRES302", "location":"EASTUS2"}
)

uploadEntity = client.upload_entities(batch=[ae])

Create an Entity (AAS Model) based on that EntityTypeDef that was created before

In [ ]:
gt = GuidTracker()
modelGuid1 = gt.get_guid()
ae = AtlasEntity(
    name="Model1",
    typeName="azure_analysis_services_model",
    qualified_name="aashomserver.azure.net/model1",
    guid=modelGuid1,
    attributes={"database-name":"AASDB1", "compatibility-level":1100}
)

uploadEntity = client.upload_entities(batch=[ae])

Create an Entity (AAS Model) based on that EntityTypeDef that was created before

In [ ]:
gt = GuidTracker()
modelGuid2 = gt.get_guid()
ae = AtlasEntity(
    name="Model2",
    typeName="azure_analysis_services_model",
    qualified_name="aashomserver.azure.net/model2",
    guid=modelGuid2,
    attributes={"database-name":"AASDB1", "compatibility-level":1100}
)

uploadEntity = client.upload_entities(batch=[ae])

In [ ]:
Create an Entity (AAS Table) based on that EntityTypeDef that was created before

In [ ]:
gt = GuidTracker()
tableGuid1 = gt.get_guid()
ae = AtlasEntity(
    name="Table1",
    typeName="azure_analysis_services_table",
    qualified_name="aashomserver.azure.net/model1/table1",
    guid=tableGuid1,
    attributes={"tableType":"Data", "query": ""}
)

uploadEntity = client.upload_entities(batch=[ae])

Create an Entity (AAS Column) based on that EntityTypeDef that was created before

In [ ]:
gt = GuidTracker()
columnGuid1 = gt.get_guid()
ae = AtlasEntity(
    name="Column1",
    description="Column description",
    typeName="azure_analysis_services_column",
    qualified_name="aashomserver.azure.net/model1/table1#column1",
    guid=columnGuid1,
    attributes={"data_type":"String", "isHidden": "FALSE", "isKey":"FALSE", "columnType":"Data"}
)

uploadEntity = client.upload_entities(batch=[ae])

Create a Relationship structure from Server to Model

In [ ]:
rd = RelationshipTypeDef(
    name="azure_analysis_services_server_models",
    attributeDefs=[],
    relationshipCategory="COMPOSITION",
    endDef1={
        "type":"azure_analysis_services_server",
        "name":"models",
        "isContainer": True,
        "cardinality":"SET",
        "isLegacyAttribute":False
    },
    endDef2={
        "type":"azure_analysis_services_model",
        "name":"server",
        "isContainer":False,
        "cardinality":"SINGLE",
        "isLegacyAttribute":False
    }
)
client.upload_typedefs(relationshipDefs=[rd])


Create a Relationship structure from Model to Table

In [ ]:
rd = RelationshipTypeDef(
    name="azure_analysis_services_model_tables",
    attributeDefs=[],
    relationshipCategory="COMPOSITION",
    endDef1={
        "type":"azure_analysis_services_model",
        "name":"tables",
        "isContainer":True,
        "cardinality":"SET",
        "isLegacyAttribute":False
    },
    endDef2={
        "type":"azure_analysis_services_table",
        "name":"model",
        "isContainer":False,
        "cardinality":"SINGLE",
        "isLegacyAttribute":False
    }
)
client.upload_typedefs(relationshipDefs=[rd])


Create a Relationship structure from Table to Column

In [ ]:
rd = RelationshipTypeDef(
    name="azure_analysis_services_table_columns",
    attributeDefs=[],
    relationshipCategory="COMPOSITION",
    endDef1={
        "type":"azure_analysis_services_table",
        "name":"columns",
        "isContainer":True,
        "cardinality":"SET",
        "isLegacyAttribute":False
    },
    endDef2={
        "type":"azure_analysis_services_column",
        "name":"table",
        "isContainer":False,
        "cardinality":"SINGLE",
        "isLegacyAttribute":False
    }
)
client.upload_typedefs(relationshipDefs=[rd])

Now create the relations between the entities, using the guid reference for the entity created before

In [ ]:
relationship = {
    "typeName": "azure_analysis_services_server_models",
    "attributes": {},
    "guid": -100,
    "provenanceType": 0,
    "end1": {
        "guid": serverGuid,
        "typeName": "azure_analysis_services_server",
        "uniqueAttributes": {"qualifiedName": "aashomserver.azure.net"}

    },
    "end2": {
        "guid": modelGuid1,
        "typeName": "azure_analysis_services_model",
        "uniqueAttributes": {"qualifiedName": "aashomserver.azure.net/model1"}
    }
}

client.upload_relationship(relationship)

In [ ]:
relationship = {
    "typeName": "azure_analysis_services_model_tables",
    "attributes": {},
    "guid": -100,
    "provenanceType": 0,
    "end1": {
        "guid": modelGuid1,
        "typeName": "azure_analysis_services_model",
        "uniqueAttributes": {"qualifiedName": "aashomserver.azure.net/model1"}
    },
    "end2": {
        "guid": tableGuid1,
        "typeName": "azure_analysis_services_table",
        "uniqueAttributes": {"qualifiedName": "aashomserver.azure.net/model1/table1"}
    }
}

client.upload_relationship(relationship)

In [ ]:
relationship = {
    "typeName": "azure_analysis_services_table_columns",
    "attributes": {},
    "guid": -100,
    "provenanceType": 0,
    "end1": {
        "guid": tableGuid1,
        "typeName": "azure_analysis_services_table",
        "uniqueAttributes": {"qualifiedName": "aashomserver.azure.net/model1/table1"}
    },
    "end2": {
        "guid": columnGuid1,
        "typeName": "azure_analysis_services_column",
        "uniqueAttributes": {"qualifiedName": "aashomserver.azure.net/model1/table1#column1"}
    }
}

client.upload_relationship(relationship)